In [1]:
import pandas as pd
import os

In [10]:
path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - statin/8. RNA-seq/Elizabeth/LW15 analysis/LW15_normalized.csv'
data = pd.read_csv(path)


In [17]:
count_matrix = data[['gene_id','gene_name','RC.1','RC.2','RC.3','H1.1','H1.2','H1.3', 'H12.1','H12.2','H12.3','RVN.1','RVN.2','RVN.3','H2.1','H2.2','H2.3']]
count_matrix.set_index('gene_id', inplace=True)

excel_file_path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/Functional screening/240122_MI_320genes_list.xlsx'
new_genes = pd.read_excel(excel_file_path, sheet_name='Sheet2')
new_genes = new_genes[['Ensembl','Name','found in']]


In [25]:
# create new functional screening dataframe
functional_screening = pd.DataFrame({'gene_ID':new_gene_list, 'gene_name':gene_names_list, 'found_in':new_genes['found in'], 'gene_regulation':None})

# fill in gene regulation column based on found_in column
functional_screening['gene_regulation'] = functional_screening['found_in'].apply(
    lambda x: 'Upregulated' if 'up' in x
    else ('Downregulated' if 'down' in x
          else None)
)

In [39]:
# read in CRISPR & cDNA library coverage data
CRISPR_path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/Functional screening/240223_Natalie_Moe requested genes Query CRISPR 891 for 274.xlsx'
cDNA_path = '/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/Functional screening/240223_Natalie_Moe requested genes Query Sigma 226 for 162.xlsx'

CRISPR_lib = pd.read_excel(CRISPR_path, sheet_name=None)
cDNA_lib = pd.read_excel(cDNA_path, sheet_name=None)

CRISPR_query = CRISPR_lib['Moe requested genes Query1']
CRISPR_coverage = CRISPR_lib['Sheet1']

cDNA_query = cDNA_lib['Moe requested genes Query']
cDNA_coverage = cDNA_lib['Gene coverage']

#add coverage data to functional_screening df
functional_screening['CRISPR_coverage'] = None
functional_screening['cDNA_coverage'] = None

CRISPR_query.set_index('Ensembl', inplace=True)
cDNA_query.set_index('Ensembl', inplace=True)

CRISPR_coverage.set_index('Gene', inplace=True)
cDNA_coverage.set_index('Moe requested genes_Name', inplace=True)

In [63]:
# fill in gene regulation column for the PCA only genes by looking in the gene count matrix

# Loop through each row to set the value in 'Regulation' column
# Loop over each row in the DataFrame using iterrows()
for index, row in functional_screening.iterrows():
    # Check if 'gene_regulation' is None
    if pd.isnull(row['gene_regulation']):
        # Retrieve the gene_id
        gene_id = row['gene_ID']

        # Calculate the averages for the specified columns
        avg_sensitive = count_matrix.loc[gene_id, ['RVN.1','RVN.2','RVN.3','H2.1','H2.2','H2.3']].mean()
        avg_resistant = count_matrix.loc[gene_id, ['RC.1','RC.2','RC.3','H1.1','H1.2','H1.3', 'H12.1','H12.2','H12.3']].mean()

        # Determine the regulation based on the averages and assign it
        if avg_sensitive > avg_resistant:
            functional_screening.at[index, 'gene_regulation'] = 'Upregulated'
        elif avg_sensitive < avg_resistant:
            functional_screening.at[index, 'gene_regulation'] = 'Downregulated'


In [15]:
genes_dict = {}
categories = new_genes['found in'].unique()
# print(categories)
# add category keys to dict
for category in categories:
    genes_dict[category] = []

# Iterate through the rows of the DataFrame
for index, row in new_genes.iterrows():
    gene = row['Ensembl']
    category = row['found in']
    genes_dict[category].append(gene)

new_gene_list = new_genes['Ensembl']

id_to_name_dict = pd.Series(gene_conversions['gene_name'].values,index=gene_conversions['gene_id']).to_dict()
gene_names_dict = {}

for category, list in genes_dict.items():
    gene_names_dict[category] = []
    for gene_id in list:
        if gene_id in id_to_name_dict:
            gene_name = id_to_name_dict.get(gene_id)
        else:
            gene_name = gene_id
            print(gene_id)
        gene_names_dict[category].append(gene_name)

gene_names_list = [item for sublist in gene_names_dict.values() for item in sublist]

In [54]:
for index, row in functional_screening.iterrows():
    gene_id = row['gene_ID']
    # check if gene is in CRISPR lib
    if gene_id in CRISPR_query.index:
        CRISPR_names = CRISPR_query.loc[gene_id, 'gene_symbol']
        # If CRISPR_names is a Series, get the unique values, otherwise just use the value
        CRISPR_name = CRISPR_names.unique()[0] if isinstance(CRISPR_names, pd.Series) else CRISPR_names
        functional_screening.at[index, 'CRISPR_coverage'] = CRISPR_coverage.loc[CRISPR_name, 'Total clones']
    else:
        CRISPR_name = None
    # check if gene is in cDNA lib
    if gene_id in cDNA_query.index:
        cDNA_names = cDNA_query.loc[gene_id, 'Moe requested genes_Name']
        # If cDNA_names is a Series, get the unique values, otherwise just use the value
        cDNA_name = cDNA_names.unique()[0] if isinstance(cDNA_names, pd.Series) else cDNA_names
        functional_screening.at[index, 'cDNA_coverage'] = cDNA_coverage.loc[cDNA_name, 'Total clones']
    else:
        cDNA_name = None
    
    if (CRISPR_name is None) and (cDNA_name is None):
        functional_screening.at[index, 'CRISPR_coverage'] = 'No coverage'
        functional_screening.at[index, 'cDNA_coverage'] = 'No coverage'
    elif CRISPR_name is None:
        functional_screening.at[index, 'CRISPR_coverage'] = 0
    elif cDNA_name is None:
        functional_screening.at[index, 'cDNA_coverage'] = 0



In [57]:
# make CRISPR_coverage column int type 
# Define a lambda function that checks if the input is numeric and converts it to int if true,
# otherwise returns 'No coverage'
convert_to_int_or_no_coverage = lambda x: int(x) if not pd.isna(pd.to_numeric(x, errors='coerce')) else 'No coverage'

# Apply this function to each entry in the 'CRISPR_coverage' column
functional_screening['CRISPR_coverage'] = functional_screening['CRISPR_coverage'].apply(convert_to_int_or_no_coverage)

In [59]:
# save functional_screening df
os.chdir('/Users/elizabeth 1/Library/CloudStorage/Box-Box/Wu Lab/Project - 2024/Functional screening')
functional_screening.to_excel('EB_functional_screening.xlsx', index=False)